# VOCS data structure 

Variables, Objectives, Constraints, and other Settings (VOCS) helps define our optimization problems. 

In [1]:
from xopt.vocs import VOCS

In [2]:
help(VOCS)

Help on class VOCS in module xopt.vocs:

class VOCS(xopt.pydantic.XoptBaseModel)
 |  VOCS(*, variables: Dict[str, types.ConstrainedListValue] = {}, constraints: Dict[str, types.ConstrainedListValue] = {}, objectives: Dict[str, xopt.vocs.ObjectiveEnum] = {}, constants: Dict[str, Any] = {}, linked_variables: Dict[str, str] = {}) -> None
 |  
 |  Variables, Objectives, Constraints, and other Settings (VOCS) data structure
 |  to describe optimization problems.
 |  
 |  Method resolution order:
 |      VOCS
 |      xopt.pydantic.XoptBaseModel
 |      pydantic.main.BaseModel
 |      pydantic.utils.Representation
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  as_yaml(self)
 |  
 |  constraint_data(self, data: Union[pandas.core.frame.DataFrame, List[Dict]], prefix: str = 'constraint_') -> pandas.core.frame.DataFrame
 |      Returns a dataframe containing constraint data transformed according to
 |      `vocs.constraints` such that values that satisfy each constraint are negat

In [3]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)
vocs

VOCS(variables={'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}, constraints={'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]}, objectives={'c': 'MAXIMIZE', 'd': 'MINIMIZE'}, constants={'g': 1234}, linked_variables={})

In [4]:
# as dict
vocs.dict()

{'variables': {'a': [0.0, 1000.0], 'b': [-1.0, 1.0]},
 'constraints': {'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]},
 'objectives': {'c': 'MAXIMIZE', 'd': 'MINIMIZE'},
 'constants': {'g': 1234},
 'linked_variables': {}}

In [5]:
#  re-parse dict
vocs2 = VOCS.parse_obj(vocs.dict())

In [6]:
# Check that these are the same
vocs2 == vocs

True

In [7]:
# This replaces the old vocs["variables"]
getattr(vocs, "variables")

{'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}

In [8]:
vocs.objectives["c"] == 'MAXIMIZE'

True

In [9]:
# json
vocs.json()

'{"variables":{"a":[0.0,1000.0],"b":[-1.0,1.0]},"constraints":{"e":["LESS_THAN",2.0],"f":["GREATER_THAN",0.0]},"objectives":{"c":"MAXIMIZE","d":"MINIMIZE"},"constants":{"g":1234},"linked_variables":{}}'

# Objective Evaluation

In [10]:
from xopt.vocs import form_objective_data, form_constraint_data, form_feasibility_data
import pandas as pd
import numpy as np

data = pd.DataFrame(vocs.random_inputs(10))
# Add some outputs
data["c"] = data["a"] + data["b"]
data["d"] = data["a"] - data["b"]
data["e"] = data["a"] * 2 + data["b"] * 2
data["f"] = data["a"] * 2 - data["b"] * 2
data.index = np.arange(len(data)) + 5  # custom index
data

,a,b,g,c,d,e,f
5,782.813651,0.238497,1234,783.052148,782.575153,1566.104296,1565.150307
6,736.063199,0.681861,1234,736.745060,735.381338,1473.490120,1470.762676
7,729.842191,-0.478220,1234,729.363971,730.320411,1458.727942,1460.640822
8,513.681440,-0.526003,1234,513.155436,514.207443,1026.310873,1028.414886
9,356.692347,0.700843,1234,357.393190,355.991504,714.786381,711.983007
10,284.072945,-0.666306,1234,283.406639,284.739252,566.813278,569.478503
11,823.031493,-0.791226,1234,822.240267,823.822719,1644.480534,1647.645439
12,524.349617,0.216744,1234,524.566361,524.132874,1049.132722,1048.265748
13,444.465805,-0.309325,1234,444.156479,444.775130,888.312958,889.550260
14,339.834978,-0.214964,1234,339.620014,340.049942,679.240028,680.099884


In [11]:
vocs.objectives

{'c': 'MAXIMIZE', 'd': 'MINIMIZE'}

In [12]:
# These are in standard form for minimization
form_objective_data(vocs.objectives, data)

,objective_c,objective_d
5,-783.052148,782.575153
6,-736.745060,735.381338
7,-729.363971,730.320411
8,-513.155436,514.207443
9,-357.393190,355.991504
10,-283.406639,284.739252
11,-822.240267,823.822719
12,-524.566361,524.132874
13,-444.156479,444.775130
14,-339.620014,340.049942


In [13]:
# This is also available as a method
vocs.objective_data(data)

,objective_c,objective_d
5,-783.052148,782.575153
6,-736.745060,735.381338
7,-729.363971,730.320411
8,-513.155436,514.207443
9,-357.393190,355.991504
10,-283.406639,284.739252
11,-822.240267,823.822719
12,-524.566361,524.132874
13,-444.156479,444.775130
14,-339.620014,340.049942


In [14]:
# use the to_numpy() method to convert for low level use.
vocs.objective_data(data).to_numpy()

array([[-783.05214793,  782.57515326],
       [-736.74505997,  735.38133789],
       [-729.36397107,  730.32041104],
       [-513.15543638,  514.20744313],
       [-357.39319046,  355.99150358],
       [-283.40663923,  284.73925153],
       [-822.24026695,  823.82271945],
       [-524.56636088,  524.13287383],
       [-444.15647923,  444.77512981],
       [-339.62001412,  340.04994209]])

In [15]:
vocs.constraint_data(data)

,constraint_e,constraint_f
5,1564.104296,-1565.150307
6,1471.490120,-1470.762676
7,1456.727942,-1460.640822
8,1024.310873,-1028.414886
9,712.786381,-711.983007
10,564.813278,-569.478503
11,1642.480534,-1647.645439
12,1047.132722,-1048.265748
13,886.312958,-889.550260
14,677.240028,-680.099884


In [16]:
vocs.feasibility_data(data)

,feasible_e,feasible_f,feasible
5,False,True,False
6,False,True,False
7,False,True,False
8,False,True,False
9,False,True,False
10,False,True,False
11,False,True,False
12,False,True,False
13,False,True,False
14,False,True,False


# Error handling

In [17]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)

In [18]:
d = {'a': [1,2,3]}

df = pd.DataFrame(d)
df2 = pd.DataFrame(df).copy()

df2['b'] = np.nan
df2['b'] - 1

0   NaN
1   NaN
2   NaN
Name: b, dtype: float64

In [19]:
data['a']  = np.nan

In [20]:
a = 2
def f(x=a):
    return x
a=99
f()

2

In [21]:
pd.DataFrame(6e66, index=[1,2,3], columns=['A'])

,A
1,6.000000e+66
2,6.000000e+66
3,6.000000e+66


In [22]:
# These are in standard form for minimization

data = pd.DataFrame({'c':[1,2,3,4]}, index=[9,3,4,5])

form_objective_data(vocs.objectives, data)

,objective_c,objective_d
9,-1.0,inf
3,-2.0,inf
4,-3.0,inf
5,-4.0,inf
